# Calculation of reflection, transmission, and absorption (RTA)

This notebook demonstrates the general use of the transfermatrix package. As an illustration, the RTA calculation of different optical thin-film systems is shown in the following.     

## Example 1: air-glass interface

In [1]:
# define optical constants and thicknesses

from pathlib import Path

materials_path = Path("materials")
glass = materials_path / "glass.csv"
air = materials_path / "air.csv"

In [2]:
# define callable that reads the optical constants

import numpy as np
from transfermatrix.input import OpticalConstants

def nk_reader(path: Path) -> OpticalConstants:
    wavelengths, n, k = np.loadtxt(path, skiprows=1, delimiter=",").T
    return OpticalConstants(wavelengths=wavelengths, n=n, k=k)

In [3]:
# define layer stack

from functools import partial
from transfermatrix.input import Layer

MyLayer = partial(Layer, nk_reader=nk_reader)

layers = [
    MyLayer(nk_file=air),
    MyLayer(nk_file=glass, thickness=1.e6),
]

In [6]:
# define generic parameters

from transfermatrix.input import Polarization

wavelengths = np.arange(400, 802, 2)
angles = np.arange(0, 90, 1)
polarization = Polarization.P

In [7]:
# calculate RTA

from transfermatrix.TransferMatrix import TransferMatrix


def get_nk_list(layers: list[Layer], wavelengths: list[float]):
    nk_list = []
    for layer in layers:
        n = np.interp(wavelengths, layer.wavelengths, layer.n)
        k = np.interp(wavelengths, layer.wavelengths, layer.k)
        nk_list.append(n + 1.j*k)
    return nk_list


tm = TransferMatrix(
    nk_list=get_nk_list(layers, wavelengths),
    thickness_list=[layer.thickness for layer in layers],
    vacuum_wavelengths_list=wavelengths,
    is_coherent_list=[layer.is_coherent for layer in layers]
)
tm.set_polarization(polarization)
tm.set_angles(angles, layer_idx=0)
tm.run_tm()

R = np.round(tm.R[0][0], 3)
T = np.round(tm.T[0][0], 3)
A = np.round(1 - R - T, 3)

print(f"R = {R}")
print(f"T = {T}")
print(f"A = {A}")

R = 0.04
T = 0.96
A = 0.0
